# ETL

Se importan las bibliotecas necesarias para el manejo de los datos, para la carga debemos tener en cuenta que los archivos se encuentran en JSON y queremos convertirlos en un Dataframe.

In [11]:
import pandas as pd
from pandas import json_normalize
import json as js
import ast as ast
import re
from textblob import TextBlob

### Carga de datos

Para el Dataset de steam_games, se pudo utilizar el pd.read para la carga de sus datos y funcionó correctamente. Los Datasets de user_reviews y user_items al contener listas anidadas en sus celdas, se tuvo que proceder con un código diferente: Se crea una lista vacía y se utiliza un bucle for para leer cada línea del archivo JSON, convertirla a un diccionario de Python y agregarla a la lista. Finalmente, la lista se convierte en un DataFrame de Pandas usando pd.DataFrame.

In [12]:
games = pd.read_json('output_steam_games.json', lines = True)

In [13]:
list_rev = []
archivo1 = r'australian_user_reviews.json'
with open(archivo1, encoding='utf-8') as file:
    for line in file.readlines():
        list_rev.append(ast.literal_eval(line))

reviews = pd.DataFrame(list_rev)

In [14]:
list_items = []
archivo2 = r'australian_users_items.json'
with open(archivo2, encoding='utf-8') as file:
    for line in file.readlines():
        list_items.append(ast.literal_eval(line))

items = pd.DataFrame(list_items)

### Desanidación de tablas

Ya tenemos las tablas cargadas, pero se puede observar que los Dataframes de Reviews e Items contienen ambos una columna con listas anidadas ('reviews' e 'items'), se procede entonces a la desanidación de estas para poder trabajar con el Dataframe de la manera más optima.

In [15]:
reviews = reviews.explode('reviews', ignore_index=True)
reviews_expanded = pd.json_normalize(reviews['reviews'])
reviews_expanded=reviews_expanded.replace('',None)
reviews = reviews.join(reviews_expanded)
reviews.drop(columns=['reviews'],inplace=True)

In [16]:
items = items.explode('items', ignore_index=True)
items_expanded = pd.json_normalize(items['items'])
items_expanded = items_expanded.replace('', None)
items = items.join(items_expanded)
items.drop(columns=['items'], inplace=True)

### Eliminación de columnas innecesarias

Después de leer todas las consultas y exigencias del proyecto a realizar, se pudo llegar a la conclusión de cuales columnas serán utiles y cuales no, basados en esto, podemos proceder a eliminar las columnas que NO van a ser de algún uso para este análisis.

In [17]:
games = games.drop(columns=['publisher', 'title', 'url', 'reviews_url','specs','early_access'])

In [18]:
reviews = reviews.drop(columns=['user_url', 'funny', 'posted', 'last_edited', 'helpful'])

In [19]:
items = items.drop(columns=['items_count', 'steam_id', 'user_url'])

### Valores nulos

El análisis se realizó con df.isnull().sum() 
- El Dataframe Games contiene 88310 filas con todas sus celdas en Nan, todas estas serán eliminadas del Dataframe; los valores faltantes de las filas restantes las cuales contienen información serán tratadas en las siguientes secciones.
- El Dataframe Reviews contiene 4 columnas y 28 filas con valores nulos en 3 de sus columnas, estas serán eliminadas ya que no nos brindan ninguna información.
- El Dataframe Items contiene 16806 filas con 4 de 5 columnas en Nan, estás serán eliminadas ya que no contienen ninguna información valiosa.


In [20]:
games = games.dropna(how='all')

In [21]:
reviews = reviews.dropna(thresh=3)

In [22]:
items = items.dropna()

### Valores faltantes y Tipo de dato

El Dataframe Games contiene valores que deben ser normalizados, corregidos y rellenados. A continuación todos los procedimientos que se le realizaron:

1. La columna Price contiene muchas celdas en string, las cuales en su mayoría deberían ser 0.00, ya que aluden a juegos que son 'Free To Play'. A excepción de 2 precios los cuales si contienen un valor especifico (499.00 y 449.00), estos serán corregidos por su valor numerico correspondiente y el resto de strings serán convertidos en 0.00. Ya teniendo todos los valores de la columna Price en tipo numerico puede ser modificado su tipo de dato.

In [23]:
games['price'] = games['price'].replace('Starting at $499.00', 499.00)
games['price'] = games['price'].replace('Starting at $449.00', 449.00)

mask = games['price'].apply(lambda x: isinstance(x, str))
games.loc[mask, 'price'] = 0.00

# Se le agrega el errors para las celdas que contienen None
games['price'] = pd.to_numeric(games['price'], errors='coerce')

2. La columna Release_date contiene la fecha de lanzamiento de los juegos, vienen muchos formatos dentro de esta que deberán ser normalizados, meses en texto, estaciones del año...Se construyo entonces una función para todos los casos detectados y poder convertir esta columna en tipo de dato fecha, para posteriormente extraer el año (Se necesita para las consultas) y cambiar el nombre de la columna por 'year'.

In [24]:
def convertir_fecha(fecha):
    # Formatos Mes año, mes año, año...
    for fmt in ['%B %Y', '%b %Y', '%Y-%m-%d', '%Y']:
        try:
            return pd.to_datetime(fecha, format=fmt)
        except ValueError:
            continue

    # Casos donde hay texto antes del año
    match = re.search(r'\b(\d{4})\b', fecha)
    if match:
        # Si encuentra un año, convertirlo
        return pd.to_datetime(match.group(1), format='%Y')

    return pd.NaT  # Si no coincide con ningún formato, devuelve NaT

# Aplicar la función de conversión
games['release_date'] = games['release_date'].apply(convertir_fecha)

# Extraer el año
games['release_date'] = games['release_date'].dt.year

# Cambio de nombre de la columna release_date
games = games.rename(columns={'release_date': 'year'})

3. Las columnas 'app_name' e 'id' serán renombradas para que coincidan con los nombres de las columnas de las tablas Reviews e Items, esto con el fin de poder hacer un 'merged' o 'join' en las consultas posteriores.

In [25]:
games = games.rename(columns={'app_name': 'item_name'})

games = games.rename(columns={'id': 'item_id'})

4. Se necesita hacer el cambio de tipo de dato a dos columnas: 'item_id' y 'year', para las consultas posteriores...pero esto requiere que no existan valores en None, por ende se tomó la decisión de convertirlos en 0.00 para posteriormente hacer el cambio.

In [26]:
games['item_id'] = games['item_id'].fillna(0)
games['item_id'] = games['item_id'].astype(int)

games['year'] = games['year'].fillna(0)
games['year'] = games['year'].astype(int)

El Dataframe Items, necesita cambiar el tipo de dato de item_id para poder hacer merged.

In [27]:
items['item_id'] = items['item_id'].astype(int)

### Valores duplicados

Después de tener los Dataframes ya limpios, se procede por último a borrar las filas exactamente iguales. Ahora los Datasets están listos para ser trabajados.

In [28]:
games = games.drop_duplicates(subset=['item_name', 'year', 'price', 'item_id', 'developer'])

In [29]:
reviews = reviews.drop_duplicates()

In [30]:
items = items.drop_duplicates()

### Análisis de Sentimiento con NLP

La función realizada aquí se centra en analizar el sentimiento sobre cada fila, basándose en dos campos: review (reseña) y recommend (si el usuario recomienda o no el juego).Si la reseña está vacía, utiliza la recomendación del usuario para determinar el sentimiento (positivo o negativo). Si la reseña está presente, usa la polaridad del texto con TextBlob para clasificar la reseña como positiva, negativa o neutral, y guarda el resultado en la columna de review.

In [31]:
def analizar_sentimiento(row):
    reseña = row['review']
    recomendacion = row['recommend']
    
    # Caso: la reseña está vacía
    if pd.isnull(reseña) or reseña.strip() == '':
        # Si no hay reseña, usar la recomendación para determinar el sentimiento
        if recomendacion:
            return 2  # Si recomienda el juego, asignar positivo
        else:
            return 0  # Si no recomienda el juego, asignar negativo
    
    # Si la reseña está presente, hacer el análisis de sentimiento
    analysis = TextBlob(reseña).sentiment.polarity
    
    # Asignar el sentimiento basado en la polaridad
    if analysis < -0.1:
        return 0  # Sentimiento negativo
    elif analysis > 0.1:
        return 2  # Sentimiento positivo
    else:
        return 1  # Neutral

# Aplicar la función al dataset
reviews['review'] = reviews.apply(analizar_sentimiento, axis=1)

### Guardar los archivos

Se instala la librería pyarrow y se guardan los archivos ya tratados y limpios en formato tipo Parquet.

In [32]:
# Ruta especifica donde quiero guardar los archivos parquet
ruta_games = 'C:\\Users\\Sarita\\Desktop\\SOY HENRY\\Proyecto 1\\games_cleaned.parquet'
ruta_reviews = 'C:\\Users\\Sarita\\Desktop\\SOY HENRY\\Proyecto 1\\reviews_cleaned.parquet'
ruta_items = 'C:\\Users\\Sarita\\Desktop\\SOY HENRY\\Proyecto 1\\items_cleaned.parquet'

# Guarda el DataFrame en formato parquet en la ruta especificada
games.to_parquet(ruta_games, index=False)
reviews.to_parquet(ruta_reviews, index=False)
items.to_parquet(ruta_items, index=False)

### Archivos Render

Ya que es una aplicación gratuita de poca memoria disponible y se necesitan probar las consultas realizadas, se hará entonces con una muestra representativa de los Dataframes.

In [63]:
# Cargar los DataFrames completos
games_sample = games.head(1000)
reviews_sample = reviews.head(1000)
items_sample = items.head(1000)

# Guardar las muestras en archivos Parquet
games_sample.to_parquet('games_sample.parquet')
reviews_sample.to_parquet('reviews_sample.parquet')
items_sample.to_parquet('items_sample.parquet')

In [37]:
games_sample.columns = games_sample.columns.str.strip()
items_sample.columns = items_sample.columns.str.strip()


In [38]:
games_sample = games_sample.drop_duplicates(subset="item_name")
items_sample = items_sample.drop_duplicates(subset="item_name")

In [39]:
games_sample = games_sample.dropna(subset=["item_name"])
items_sample = items_sample.dropna(subset=["item_name"])


In [52]:
if 'item_name' in games_sample.columns and 'item_name' in items_sample.columns:
    df_merged = pd.merge(games_sample, items_sample, on='item_name', how='inner')
else:
    raise KeyError("La columna 'item_name' no está presente en uno de los DataFrames.")

In [41]:
print(games_sample.columns)
print(items_sample.columns)

Index(['genres', 'item_name', 'year', 'tags', 'price', 'item_id', 'developer'], dtype='object')
Index(['user_id', 'item_id', 'item_name', 'playtime_forever',
       'playtime_2weeks'],
      dtype='object')


In [42]:
print(games_sample['item_name'].dtype)
print(items_sample['item_name'].dtype)

object
object


In [43]:
games_sample['item_name'] = games_sample['item_name'].str.strip()
items_sample['item_name'] = items_sample['item_name'].str.strip()

In [44]:
print(games_sample['item_name'].isnull().sum())
print(items_sample['item_name'].isnull().sum())

0
0


In [45]:
df_merged = pd.merge(games_sample, items_sample, on='item_name', how='inner')

In [46]:
print(games_sample.columns)
print(items_sample.columns)

Index(['genres', 'item_name', 'year', 'tags', 'price', 'item_id', 'developer'], dtype='object')
Index(['user_id', 'item_id', 'item_name', 'playtime_forever',
       'playtime_2weeks'],
      dtype='object')


In [47]:
print(games_sample[['item_name']].head())
print(items_sample[['item_name']].head())

                                                item_name
93213                       HELLDIVERS™ - Specialist Pack
100344                             White Noise 2 - Lilith
94498        AdVenture Capitalist - Savvy Investor Bundle
94382   Rocksmith® 2014 – Rise Against - “Help Is On T...
95766                                     Desperate Times
                              item_name
1784637  The Elder Scrolls IV: Oblivion
1476866                       Grow Home
2619348                    King's Quest
4595311                       theHunter
3993358         Half-Life 2: Deathmatch


In [48]:
df_test = pd.merge(games_sample, items_sample, on='item_name', how='outer')
print(df_test.head())

                                              genres  \
0  [Action, Free to Play, Indie, Massively Multip...   
1                                                NaN   
2                         [Action, Adventure, Indie]   
3                                                NaN   
4                                                NaN   

                                item_name    year  \
0     "Barely Attuned Magic Thingy" Staff  2014.0   
1  "Glow Ball" - The billiard puzzle game     NaN   
2                                    .EXE  2016.0   
3                               0RBITALIS     NaN   
4           1 Moment Of Time: Silentville     NaN   

                                                tags  price  item_id_x  \
0  [Action, Free to Play, RPG, Massively Multipla...    NaN   308163.0   
1                                                NaN    NaN        NaN   
2  [Adventure, Indie, Action, Platformer, Rogue-l...   5.99   471640.0   
3                                               

In [53]:
print("Unique item_names in games_sample:")
print(games_sample['item_name'].unique())

print("Unique item_names in items_sample:")
print(items_sample['item_name'].unique())

Unique item_names in games_sample:
['Lost Summoner Kitty' 'Ironbound' 'Real Pool 3D - Poolians' '弹炸人2222'
 'Log Challenge' 'Battle Royale Trainer' 'SNOW - All Access Basic Pass'
 'SNOW - All Access Pro Pass' 'SNOW - All Access Legend Pass' 'Race'
 'Uncanny Islands' 'Icarus Six Sixty Six'
 'Army of Tentacles: (Not) A Cthulhu Dating Sim: Black GOAT of the Woods Edition'
 'Beach Rules' 'Planetarium 2 - Zen Odyssey' 'The first thrust of God'
 'SNOW - Starter Pack' 'The first thrust of God - All Aircrafts'
 'SNOW - Lifetime Pack' 'After Life VR' 'Kitty Hawk'
 'The Warrior Of Treasures' 'Mortars VR' 'RC Plane 3 - Canyon Scenario'
 'RC Plane 3 - F-53B' 'DinoBlaster'
 'Robotpencil Presents: Understanding 3D for Concept'
 'Carmageddon Max Pack' 'Half-Life' 'Fallen Mage'
 "Don't Get Hit In The Face" 'Lost Cities' 'Twisted Enhanced Edition'
 'Vaporwave Simulator' 'Tactics Forever' 'Christmas Santa Troubles'
 'Kingdom Watcher' 'Underdone - Soundtrack' 'Bitcoin Clicker'
 "Disciples II: Gallean's Re

In [50]:
not_in_items_sample = games_sample[~games_sample['item_name'].isin(items_sample['item_name'])]
not_in_games_sample = items_sample[~items_sample['item_name'].isin(games_sample['item_name'])]

print("Items in games_sample not in items_sample:")
print(not_in_items_sample)

print("Items in items_sample not in games_sample:")
print(not_in_games_sample)

Items in games_sample not in items_sample:
                                           genres  \
93213                                    [Action]   
100344               [Adventure, Indie, Strategy]   
94498   [Casual, Free to Play, Indie, Simulation]   
94382                        [Casual, Simulation]   
95766                             [Action, Indie]   
...                                           ...   
99592                             [Action, Indie]   
106422                                [Adventure]   
105109                          [Indie, Strategy]   
101430                                 [Strategy]   
110765                  [Action, Indie, Strategy]   

                                                item_name  year  \
93213                       HELLDIVERS™ - Specialist Pack  2015   
100344                             White Noise 2 - Lilith  2017   
94498        AdVenture Capitalist - Savvy Investor Bundle  2016   
94382   Rocksmith® 2014 – Rise Against - “Help Is On

In [54]:
games_sample[games_sample['item_name']=='Grand Theft Auto III']

,genres,item_name,year,tags,price,item_id,developer
88866,[Action],Grand Theft Auto III,2002,"[Open World, Action, Classic, Third Person, Si...",9.99,12100,Rockstar Games


In [64]:
df_merged = pd.merge(games_sample, items_sample, on='item_name', how='inner')


In [70]:
print(df_merged.columns)


Index(['genres', 'item_name', 'year', 'tags', 'price', 'item_id', 'developer',
       'user_id', 'playtime_forever', 'playtime_2weeks'],
      dtype='object')


In [58]:

# Elimina la columna que no necesitas
df_merged = df_merged.drop(columns=['item_id_y'])

# Renombra la columna que quedará
df_merged.rename(columns={'item_id_x': 'item_id'}, inplace=True)

# Muestra el resultado final
print(df_merged.head())


               genres                                          item_name  \
0                 NaN  1... 2... 3... KICK IT! (Drop That Beat Like a...   
1                 NaN                                  100% Orange Juice   
2  [Adventure, Indie]                                          1000 Amps   
3            [Racing]             18 Wheels of Steel: American Long Haul   
4        [Simulation]                18 Wheels of Steel: Extreme Trucker   

     year                                               tags  price   item_id  \
0     NaN                                                NaN    NaN       NaN   
1     NaN                                                NaN    NaN       NaN   
2  2012.0  [Indie, Platformer, Puzzle, Adventure, Metroid...   4.99  205690.0   
3  2007.0          [Simulation, Racing, Driving, Open World]   9.99   12520.0   
4  2009.0                              [Simulation, Driving]   9.99   33730.0   

        developer            user_id  playtime_forever  

In [ ]:
# Eliminar duplicados en la columna item_name
df = games_sample.drop_duplicates(subset='item_name')
df2 = items_sample.drop_duplicates(subset='item_name')

# Realizar el merge
df_merged = pd.merge(games_sample, items_sample, on=['item_name', 'item_id'], how='inner')


In [68]:
# Realizar el merge
df_merged = pd.merge(games_sample, items_sample, on=['item_name', 'item_id'], how='inner')

In [69]:
print(df_merged)

                            genres                item_name  year  \
0                         [Action]                Half-Life  1998   
1                         [Action]                Half-Life  1998   
2                [Indie, Strategy]                   DEFCON  2006   
3              [Indie, Simulation]              Garry's Mod  2006   
4                            [RPG]              Arx Fatalis  2002   
..                             ...                      ...   ...   
107  [Indie, Simulation, Strategy]  Gratuitous Tank Battles  2012   
108      [Casual, Indie, Strategy]           Ticket to Ride  2012   
109                       [Action]              Max Payne 3  2012   
110                [Indie, Casual]                   Splice  2012   
111                        [Indie]    Indie Game: The Movie  2012   

                                                  tags  price  item_id  \
0    [FPS, Classic, Action, Sci-fi, Singleplayer, S...   9.99       70   
1    [FPS, Classic, Act

In [66]:
games_sample

,genres,item_name,year,tags,price,item_id,developer
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018,"[Strategy, Action, Indie, Casual, Simulation]",4.99,761140,Kotoshiro
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018,"[Free to Play, Strategy, Indie, RPG, Card Game...",0.00,643980,Secret Level SRL
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017,"[Free to Play, Simulation, Sports, Casual, Ind...",0.00,670290,Poolians.com
88313,"[Action, Adventure, Casual]",弹炸人2222,2017,"[Action, Adventure, Casual]",0.99,767400,彼岸领域
88314,None,Log Challenge,0,"[Action, Indie, Casual, Sports]",2.99,773570,None
...,...,...,...,...,...,...,...
89305,[Strategy],Magic 2013 “Pack Instinct” Deck Key,2012,[Strategy],0.99,97364,Stainless Games
89306,[Action],Street Fighter X Tekken: Hugo (Swap Costume),2012,[Action],0.99,210041,"Capcom U.S.A., Inc."
89307,[Strategy],Magic 2013 “Obedient Dead” Foil Conversion,2012,[Strategy],0.99,97371,Stainless Games
89308,[Strategy],Magic 2013 “Goblin Gangland” Foil Conversion,2012,[Strategy],0.99,97369,Stainless Games


In [67]:
items

,user_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,10,Counter-Strike,6.0,0.0
1,76561197970982479,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,30,Day of Defeat,7.0,0.0
3,76561197970982479,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...
5170009,76561198329548331,346330,BrainBread 2,0.0,0.0
5170010,76561198329548331,373330,All Is Dust,0.0,0.0
5170011,76561198329548331,388490,One Way To Die: Steam Edition,3.0,3.0
5170012,76561198329548331,521570,You Have 10 Seconds 2,4.0,4.0
